# Decision Trees

A `decision tree` is a predictive modeling tool which uses a tree structure to represent a number of possible `decision paths` and an outcome for each path.

<img src="images/tree1.png" alt="" style="width: 600px;"/>


Decision trees have a lot to recommend them. They’re `very easy to understand and interpret`, and the process by which they reach a prediction is completely transparent. Unlike the other models, decision trees `can easily handle a mix of numeric` (e.g., number of legs) `and categorical` (e.g., delicious/not delicious) `attributes and can even classify data for which attributes are missing`.

At the same time, finding an “optimal” decision tree for a set of training data is computationally a very hard problem. (We will get around this by trying to build a goodenough tree rather than an optimal one, although for large data sets this can still be a lot of work.) More important, `it is very easy (and very bad) to build decision trees that are overfitted to the training data`, and that don’t generalize well to unseen data. We’ll look at ways to address this.

Most people divide decision trees into `classification trees` (which produce categorical outputs) and `regression trees` (which produce numeric outputs). 

## Entropy
In order to build a `decision tree`, we will need to decide what questions to ask and in what order. At each stage of the tree there are some possibilities we’ve eliminated and some that we haven’t. Every possible question partitions the remaining possibilities according to their answers.

## Example 1: Identify which candidates will interview well